## Setting up google colab

In [5]:
from google.colab import drive    #when not on google colab, comment out these lines
drive.mount('/content/drive')     #when not on google colab, comment out these lines

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys                                                                                     #when not on google colab, comment out these lines
sys.path.append('/content/drive/My Drive/midas/competitive-data-science-predict-future-sales') 

## Importing Libraries

In [0]:
import numpy as np
import pandas as pd 
import os
import random as rn
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from itertools import product
from sklearn.preprocessing import LabelEncoder

## Loading Dataset

In [0]:
train = pd.read_csv('/content/drive/My Drive/midas/competitive-data-science-predict-future-sales/sales_train_v2.csv')
test = pd.read_csv('/content/drive/My Drive/midas/competitive-data-science-predict-future-sales/test.csv')
items = pd.read_csv('/content/drive/My Drive/midas/competitive-data-science-predict-future-sales/items.csv')
cats = pd.read_csv('/content/drive/My Drive/midas/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('/content/drive/My Drive/midas/competitive-data-science-predict-future-sales/shops.csv')
sample_submission = pd.read_csv('/content/drive/My Drive/midas/competitive-data-science-predict-future-sales/sample_submission.csv')

Removing any outliers in item price and in item  number of products sold.

In [0]:
train = train[train.item_price<100000]
train = train[train.item_cnt_day<1001]

Replacing negative values in the dataset by median.

In [0]:
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median

Several shops are duplicates of each other (according to their name). Fixing the train and test set.

In [0]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

Creating additional features based on the following observations
* Each shop_name starts with the city name.
* Each category contains type and subtype in its name.

In [0]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
# if subtype is nan then type
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code', 'subtype_code']]

items.drop(['item_name'], axis=1, inplace=True)

Rearranging the training dataset to get the proper format so as to make it similar to the test dataset.

In [0]:
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = train[train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
    
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num']
matrix['shop_id'] = matrix['shop_id']
matrix['item_id'] = matrix['item_id']
matrix.sort_values(cols,inplace=True)

In [14]:
matrix.head()

,date_block_num,shop_id,item_id
114910,0,2,19
117150,0,2,27
120623,0,2,28
118316,0,2,29
114602,0,2,32


In [0]:
train['revenue'] = train['item_price'] *  train['item_cnt_day']

Aggregate train set by shop/item pairs to calculate target aggreagates, then clip(0,20) target value. This way train target will be similar to the test predictions.

In [0]:
group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

In [0]:
matrix = pd.merge(matrix, group, on=cols, how='left')

In [18]:
matrix.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,2,19,NaN
1,0,2,27,1.0
2,0,2,28,NaN
3,0,2,29,NaN
4,0,2,32,NaN


In [0]:
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)
                                .clip(0,20) # NB clip target here
                                )

In [20]:
matrix.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,2,19,0.0
1,0,2,27,1.0
2,0,2,28,0.0
3,0,2,29,0.0
4,0,2,32,0.0


Making the test dataset as an extension of the training dataset for ease of use. Here the test dataset is considered as the 34th month in the time series data.

In [0]:
test['date_block_num'] = 34

In [0]:
matrix = pd.concat([matrix, test], ignore_index=True, sort=False, keys=cols)

In [23]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month,ID
0,0,2,19,0.0,NaN
1,0,2,27,1.0,NaN
2,0,2,28,0.0,NaN
3,0,2,29,0.0,NaN
4,0,2,32,0.0,NaN
...,...,...,...,...,...
11127999,34,45,18454,NaN,214195.0
11128000,34,45,16188,NaN,214196.0
11128001,34,45,15757,NaN,214197.0
11128002,34,45,19648,NaN,214198.0


In [0]:
matrix.fillna(0, inplace=True)

In [25]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month,ID
0,0,2,19,0.0,0.0
1,0,2,27,1.0,0.0
2,0,2,28,0.0,0.0
3,0,2,29,0.0,0.0
4,0,2,32,0.0,0.0
...,...,...,...,...,...
11127999,34,45,18454,0.0,214195.0
11128000,34,45,16188,0.0,214196.0
11128001,34,45,15757,0.0,214197.0
11128002,34,45,19648,0.0,214198.0


Adding city_code, item_category_id, type_code and subtype_code as additional features.

In [0]:
matrix = pd.merge(matrix, shops, on=['shop_id'], how='left')

In [0]:
matrix = pd.merge(matrix, items, on=['item_id'], how='left')

In [0]:
matrix = pd.merge(matrix, cats, on=['item_category_id'], how='left')

In [30]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month,ID,city_code,item_category_id,type_code,subtype_code
0,0,2,19,0.0,0.0,0,40,11,4
1,0,2,27,1.0,0.0,0,19,5,10
2,0,2,28,0.0,0.0,0,30,8,55
3,0,2,29,0.0,0.0,0,23,5,16
4,0,2,32,0.0,0.0,0,40,11,4
...,...,...,...,...,...,...,...,...,...
11127999,34,45,18454,0.0,214195.0,20,55,13,2
11128000,34,45,16188,0.0,214196.0,20,64,14,42
11128001,34,45,15757,0.0,214197.0,20,55,13,2
11128002,34,45,19648,0.0,214198.0,20,40,11,4


Instead of simply using date_block_num , it is important to identify the month for better understanding of the sales.

In [0]:
matrix['month'] = matrix['date_block_num'] % 12

In [32]:
matrix.tail()

,date_block_num,shop_id,item_id,item_cnt_month,ID,city_code,item_category_id,type_code,subtype_code,month
11127999,34,45,18454,0.0,214195.0,20,55,13,2,10
11128000,34,45,16188,0.0,214196.0,20,64,14,42,10
11128001,34,45,15757,0.0,214197.0,20,55,13,2,10
11128002,34,45,19648,0.0,214198.0,20,40,11,4,10
11128003,34,45,969,0.0,214199.0,20,37,11,1,10


Observing the max number of days in the respective months.

In [0]:
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days)

In [34]:
matrix.head()

,date_block_num,shop_id,item_id,item_cnt_month,ID,city_code,item_category_id,type_code,subtype_code,month,days
0,0,2,19,0.0,0.0,0,40,11,4,0,31
1,0,2,27,1.0,0.0,0,19,5,10,0,31
2,0,2,28,0.0,0.0,0,30,8,55,0,31
3,0,2,29,0.0,0.0,0,23,5,16,0,31
4,0,2,32,0.0,0.0,0,40,11,4,0,31


## Getting ready for model training

In [0]:
data = matrix

Validation strategy is 34th month for the test set, 33rd month for the validation set and 1-33 months for the train.

In [0]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

## Defining model

In [0]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

## Training

In [38]:
model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[19:33:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:33:44] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-rmse:1.16767	validation_1-rmse:1.14718
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:1.16257	validation_1-rmse:1.13545
[2]	validation_0-rmse:1.15814	validation_1-rmse:1.12504
[3]	validation_0-rmse:1.15408	validation_1-rmse:1.11662
[4]	validation_0-rmse:1.14781	validation_1-rmse:1.10851
[5]	validation_0-rmse:1.14319	validation_1-rmse:1.10377
[6]	validation_0-rmse:1.14146	validation_1-rmse:1.09947
[7]	validation_0-rmse:1.13887	validation_1-rmse:1.09419
[8]	validation_0-rmse:1.13879	validation_1-rmse:1.08

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=300, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.8, verbosity=1)

## Predictions and submission

In [0]:
Y_test = model.predict(X_test).clip(0, 20)

In [0]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})

In [0]:
submission.to_csv('/content/drive/My Drive/midas/competitive-data-science-predict-future-sales/submission3.csv', index=False)

In [42]:
submission

,ID,item_cnt_month
0,0,0.436563
1,1,0.352290
2,2,0.159247
3,3,0.143570
4,4,0.141076
...,...,...
214195,214195,0.112115
214196,214196,0.078722
214197,214197,0.119150
214198,214198,0.094618
